In [1]:
import numpy as np
import glob
from astropy.io import fits
%pylab inline --no-import-all
plt.rc('font', family='serif', serif='Times',size=20)
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=20)
plt.rc('xtick.major', size=10)
plt.rc('ytick.major', size=10)
plt.rc('ytick', labelsize=20)
plt.rc('axes', labelsize=20)
plt.rc('figure',titlesize=20)
plt.rc('image',origin='lower',interpolation='nearest')

import sys
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
os.chdir('/Users/mrizzo/IFS/crispy/crispy/PISCES/')

import logging as log
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.tools.image import Image
from params import Params
par = Params()
par.hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT                                                                         
COMMENT ************************************************************            
COMMENT ********************** General parameters ******************            
COMMENT ************************************************************            
COMMENT                                                                         
NLENS   =                  108 / # lenslets across array                        
PITCH   =             0.000174 / Lenslet pitch (meters)                         
INTERLAC=                    2 / Interlacing                                    
PHILENS =    26.565051177077

In [3]:
print (par.wavecalDir)

..//ReferenceFiles/Calibra_20190128/


In [9]:
flist = os.listdir(par.wavecalDir)
wavs = []
for f in flist:
    tmp = f.split('.fits')[0]
    tmp = float(tmp.split('det')[-1])
    wavs.append(tmp)
wavs.sort()
print(wavs)
par.lamlist = wavs
par.filelist = [par.wavecalDir+'det'+str(wav)+'.fits' for wav in par.lamlist]
print(par.filelist)

[691.6, 696.2, 700.7, 705.3, 709.8, 714.4, 718.9, 723.5, 728.0, 732.6, 737.1, 741.7, 746.2, 750.8, 755.3, 759.9, 764.5, 769.0, 773.6, 778.1, 782.7, 787.2, 791.8, 796.3, 800.9, 805.4, 810.0, 814.5, 819.1, 823.6, 828.2]
['..//ReferenceFiles/Calibra_20190128/det691.6.fits', '..//ReferenceFiles/Calibra_20190128/det696.2.fits', '..//ReferenceFiles/Calibra_20190128/det700.7.fits', '..//ReferenceFiles/Calibra_20190128/det705.3.fits', '..//ReferenceFiles/Calibra_20190128/det709.8.fits', '..//ReferenceFiles/Calibra_20190128/det714.4.fits', '..//ReferenceFiles/Calibra_20190128/det718.9.fits', '..//ReferenceFiles/Calibra_20190128/det723.5.fits', '..//ReferenceFiles/Calibra_20190128/det728.0.fits', '..//ReferenceFiles/Calibra_20190128/det732.6.fits', '..//ReferenceFiles/Calibra_20190128/det737.1.fits', '..//ReferenceFiles/Calibra_20190128/det741.7.fits', '..//ReferenceFiles/Calibra_20190128/det746.2.fits', '..//ReferenceFiles/Calibra_20190128/det750.8.fits', '..//ReferenceFiles/Calibra_20190128/de

In [13]:
# We need to run the calibration for the first time
from crispy.tools.wavecal import buildcalibrations
par.gaussian_hires=False # since par.gaussian_hires is False, it will attempt to make high-resolution PSFLets
buildcalibrations(par,
                    inspect=True,
                    inspect_first=True,
                    genwavelengthsol=True, # Compute wavelength at the center of all pixels
                    makehiresPSFlets=True, # this requires very high SNR on the monochromatic frames
                    makePolychrome=True,   # This is needed to use least squares extraction
                    makePSFWidths=True,    # Fit PSF widths from high-res PSFLet models
                    upsample=3,            # upsampling factor of the high-resolution PSFLets
                    nsubarr=4,             # the detector is divided into nsubarr^2 regions for PSFLet averaging
                    apodize=False,         # to match PSFlet spot locations, only use the inner circular part of the 
                                           # detector, hence discarding the corners of the detector where lenslets are 
                                           # distorted
                    finecal=True,          # Attempt at doing fine wavelength calibration (need high SNR!)
                    trimfrac=0.1,
                  ) 

crispy - INFO - Building calibration files, placing results in ..//ReferenceFiles/Calibra_20190128/
crispy - INFO - Read data from HDU 0 of ..//ReferenceFiles/Calibra_20190128/det691.6.fits
crispy - INFO - Implementing experimental fine calibration method - watch out for bugs!
crispy - INFO - Read data from HDU 0 of ..//ReferenceFiles/Calibra_20190128/det691.6.fits
crispy - INFO - Mean, median, std: (2664.4119208946427, 2650.0, 69.55710303703204)
crispy - INFO - Initializing PSFlet location transformation coefficients
crispy - INFO - Performing initial optimization of PSFlet location transformation coefficients for frame ..//ReferenceFiles/Calibra_20190128/det691.6.fits
crispy - INFO - Array origin: (521.5650777734378, 523.6323690197485)
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ..//ReferenceFiles/Calibra_20190128/det691.6.fits
crispy - INFO - Array origin: (522.314594951993, 524.8730846759098)
crispy - INFO - Finding individ

KeyboardInterrupt: 

In [11]:
from crispy.tools.reduction import calculateWaveList
lam_midpts,lam_endpts = calculateWaveList(par,method='lstsq')
print('Wavelengths at midpoints for lstsq (nm): ',lam_midpts)
print('Wavelengths at endpoints for lstsq (nm): ',lam_endpts)
lam_midpts,lam_endpts = calculateWaveList(par,method='optext')
print('Wavelengths at midpoints for optext (nm): ',lam_midpts)
print('Wavelengths at endpoints for optext (nm): ',lam_endpts)

crispy - INFO - Reduced cube will have 25 wavelength bins
('Wavelengths at midpoints for lstsq (nm): ', array([694.09767464, 699.12011712, 704.17890165, 709.27429119,
       714.40655062, 719.57594672, 724.78274821, 730.02722575,
       735.30965197, 740.63030146, 745.98945079, 751.38737856,
       756.82436536, 762.30069381, 767.8166486 , 773.37251645,
       778.96858618, 784.60514868, 790.28249695, 796.00092612,
       801.76073344, 807.56221833, 813.40568236, 819.29142929,
       825.21976507]))
('Wavelengths at endpoints for lstsq (nm): ', array([691.6       , 696.60436949, 701.64495024, 706.72200429,
       711.83579554, 716.98658984, 722.17465492, 727.40026048,
       732.66367816, 737.96518157, 743.30504628, 748.68354988,
       754.10097196, 759.55759413, 765.05370004, 770.58957539,
       776.16550795, 781.78178757, 787.43870619, 793.13655789,
       798.87563885, 804.65624739, 810.47868402, 816.34325139,
       822.25025436, 828.2       ]))
crispy - INFO - Reduced cube will 

In [14]:
from crispy.IFS import reduceIFSMap
cube = reduceIFSMap(par,par.wavecalDir+'/det750.8.fits',method='sum',smoothbad=False,medsub=False)


crispy - INFO - Read data from HDU 0 of ..//ReferenceFiles/Calibra_20190128//det750.8.fits
crispy - INFO - Mean, median, std: (2791.063817853376, 2718.0, 205.08057983725396)
crispy - INFO - Reduced cube will have 25 wavelength bins
crispy - INFO - Elapsed time: 2.780445s


In [28]:
from crispy.wavecal import fit_monochromatic_cube
popt,pcov = fit_monochromatic_cube(cube.data,
                        lam_midpts,
                      returnAll=True)

print np.sqrt(pcov[1][1])
print popt[1]

0.1102917552298997
750.3997682816271
